In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
# Load tidied dataframes

# spells_df = pd.read_pickle('data/spells_df')
monster_df = pd.read_pickle('data/monster_df')
actions_df = pd.read_pickle('data/actions_df')
spellbook_df = pd.read_pickle('data/spellbook_df')
innate_spellbook_df = pd.read_pickle('data/innate_spellbook_df')
# special_abilities_df = pd.read_pickle('data/special_abilities_df')
# condition_immunities_df = pd.read_pickle('data/condition_immunities_df')
# damage_immunities_df = pd.read_pickle('data/damage_immunities_df')
# damage_resistances_df = pd.read_pickle('data/damage_resistances_df')
# damage_vulnerabilities_df = pd.read_pickle('data/damage_vulnerabilities_df')

# Extracting features from the `actions_df` dataframe

## Multiattacks

In [3]:
actions_df.sort_index(level=['name', 'action'], inplace=True)

The following creatures have `multiattack` actions which are formatted differently then the others.

In [4]:
actions_df.at[('Animated Armor', 'Multiattack'), 'desc'] = \
    'The armor makes two slam attacks.'
actions_df.at[('Cult Fanatic', 'Multiattack'), 'desc'] = \
    'The fanatic makes two melee dagger attacks.'
actions_df.at[('Deva', 'Multiattack'), 'desc'] = \
    'The deva makes two mace attacks.'
actions_df.at[('Doppelganger', 'Multiattack'), 'desc'] = \
    'The doppelganger makes two slam attacks.'
actions_df.at[('Erinyes', 'Multiattack'), 'desc'] = \
    'The erinyes makes three attacks, either with its longsword or its longbow.'
actions_df.at[('Gladiator', 'Multiattack'), 'desc'] = \
    'The gladiator makes three spear melee attacks or two ranged spear attacks.'
actions_df.at[('Iron Golem', 'Multiattack'), 'desc'] = \
    'The golem makes two melee attacks: either with its sword or its slam attack.'
actions_df.at[('Knight', 'Multiattack'), 'desc'] = \
    'The knight makes two greatsword attacks.'
actions_df.at[('Planetar', 'Multiattack'), 'desc'] = \
    'The planetar makes two greatsword attacks.'
actions_df.at[('Scout', 'Multiattack'), 'desc'] = \
    'The scout makes two shortsword attacks or two longbow attacks.'
actions_df.at[('Spy', 'Multiattack'), 'desc'] = \
    'The spy makes two shortsword attacks.'
actions_df.at[('Thug', 'Multiattack'), 'desc'] = \
    'The thug makes two mace attacks.'

In [5]:
df = actions_df.reset_index('action')

In [6]:
numerics = {'one': 1,
            'two': 2,
            'twice': 2,
            'three': 3,
            'four': 4,
            'six': 6,
            'seven': 7}
exceptions = ('or', 'alternatively', 'when', 'in place', 'replace',
              'also', 'as many', 'only', r'\dd\d', 'with a different weapon')


def normalize_multiattack(x):
    df = x.set_index('action')
    if 'Multiattack' not in df.index:
        return
    desc = df.loc['Multiattack'].desc.lower().strip()
    actions = [format_action(action) for action in df.index
               if action != 'Multiattack']
    multiattack, exception = extract_attacks(desc, actions)
    return pd.DataFrame({'desc': [desc],
                         'multiattack': [[multiattack]],
                         'exception': [exception]})


def format_action(action):
    action = action.lower()
    i = action.find('(')
    if i > 0:
        return action[:i-1]
    else:
        return action


def extract_attacks(desc, actions):
    exceptions_pattern = r'\b({})\b'.format('|'.join(x for x in exceptions))
    exception = re.search(exceptions_pattern, desc) is not None
    actions_pattern = '|'.join(x for x in actions)
    split_pattern = r'.*?({})'.format(actions_pattern)
    numerics_pattern = '|'.join(x for x in numerics.keys())
    multiattack = list()
    for x in re.finditer(split_pattern, desc):
        start, end = x.span()
        x = re.findall(numerics_pattern, desc[start:end])
        if x:
            n = numerics[x[-1]]
        else:
            n = 1
        y = re.findall(actions_pattern, desc[start:end])
        multiattack.extend(n * y)
    return multiattack, exception

In [7]:
multiattacks_df = df.groupby('name').apply(normalize_multiattack)
multiattacks_df.reset_index(level=1, drop=True, inplace=True)

We'll correct the following exceptions by hand:

In [8]:
exceptional_multiattacks = [x[0] for x in multiattacks_df[multiattacks_df.exception].iterrows()]

In [9]:
multiattacks_df.at['Bandit Captain', 'multiattack'] = [['scimitar', 'scimitar', 'dagger'],
                                                       ['dagger', 'dagger']]
multiattacks_df.at['Barbed Devil', 'multiattack'] = [['tail', 'claw', 'claw'],
                                                      ['hurl flame', 'hurl flame']]
multiattacks_df.at['Centaur', 'multiattack'] = [['pike', 'hooves'],
                                                 ['longbow', 'longbow']]
multiattacks_df.at['Chimera', 'multiattack'] = [['bite', 'horns', 'claws'],
                                                 ['fire breath', 'horns', 'claws'],
                                                 ['bite', 'fire breath', 'claws']]
multiattacks_df.at['Chuul', 'multiattack'] = [['pincer', 'pincer', 'tentacles'],
                                               ['pincer', 'pincer']]
multiattacks_df.at['Dragon Turtle', 'multiattack'] = [['bite', 'claw', 'claw'],
                                                       ['bite', 'tail']]
multiattacks_df.at['Drider', 'multiattack'] = [['longsword', 'longsword', 'longsword'],
                                                ['longsword', 'longsword', 'bite'],
                                                ['longbow', 'longbow', 'longbow'],
                                                ['longbow', 'longbow', 'bite']]
multiattacks_df.at['Efreeti', 'multiattack'] = [['scimitar', 'scimitar'],
                                                 ['hurl flame', 'hurl flame']]
multiattacks_df.at['Erinyes', 'multiattack'] = [['longsword', 'longsword', 'longsword'],
                                                ['longbow', 'longbow', 'longbow']]
multiattacks_df.at['Glabrezu', 'multiattack'] = [['pincer', 'pincer', 'fist', 'fist'],
                                                 ['pincer', 'pincer']] # spells?
multiattacks_df.at['Gladiator', 'multiattack'] = [['spear', 'spear', 'spear'],
                                                  ['spear', 'spear']]
multiattacks_df.at['Half-Red Dragon Veteran', 'multiattack'] = [['longsword', 'longsword'],
                                                                ['longsword', 'longsword', 'shortsword']]
multiattacks_df.at['Horned Devil', 'multiattack'] = [['fork', 'fork', 'tail'],
                                                     ['fork', 'hurl flame', 'tail'],
                                                     ['fork', 'fork', 'hurl flame']]
multiattacks_df.at['Hydra', 'multiattack'] = [['bite', 'bite', 'bite', 'bite', 'bite']]
multiattacks_df.at['Iron Golem', 'multiattack'] = [['sword', 'sword'],
                                                   ['slam', 'slam']]
multiattacks_df.at['Kraken', 'multiattack'] = [['tentacle', 'tentacle', 'tentacle'],
                                               ['tentacle', 'tentacle', 'fling'],
                                               ['tentacle', 'fling', 'fling'],
                                               ['fling', 'fling', 'fling']]
multiattacks_df.at['Lamia', 'multiattack'] = [['claws', 'dagger'],
                                              ['claws', 'intoxicating touch']]
multiattacks_df.at['Lizardfolk', 'multiattack'] = [['bite', 'heavy club'],
                                                   ['bite', 'javelin'],
                                                   ['bite', 'spiked shield'],
                                                   ['heavy club', 'javelin'],
                                                   ['heavy club', 'spiked shield']]
multiattacks_df.at['Manticore', 'multiattack'] = [['bite', 'claw', 'claw'],
                                                  ['tail spike', 'tail spike', 'tail spike']]
multiattacks_df.at['Medusa', 'multiattack'] = [['snake hair', 'shortsword', 'shortsword'],
                                               ['longbow', 'longbow']]
multiattacks_df.at['Merrow', 'multiattack'] = [['bite', 'claws'],
                                               ['bite', 'harpoon']]
multiattacks_df.at['Oni', 'multiattack'] = [['claw', 'claw'],
                                            ['glaive', 'glaive']]
multiattacks_df.at['Sahuagin', 'multiattack'] = [['bite', 'claws'],
                                                 ['bite', 'spear']]
multiattacks_df.at['Scout', 'multiattack'] = [['shortsword', 'shortsword'],
                                              ['longbow', 'longbow']]
multiattacks_df.at['Shambling Mound', 'multiattack'] = [['slam', 'slam'],
                                                        ['slam', 'slam', 'engulf']]
multiattacks_df.at['Vampire Spawn', 'multiattack'] = [['claws', 'claws'],
                                                      ['claws', 'bite']]
multiattacks_df.at['Violet Fungus', 'multiattack'] = [['rotting touch', 'rotting touch',
                                                       'rotting touch', 'rotting touch']]
multiattacks_df.at['Veteran', 'multiattack'] = [['longsword', 'longsword', 'shortsword'],
                                                ['longsword', 'longsword']]
multiattacks_df.at['Werebear', 'multiattack'] = [['claw', 'claw'],
                                                 ['greataxe', 'greataxe']]
multiattacks_df.at['Wight', 'multiattack'] = [['longsword', 'longsword'],
                                              ['longbow', 'longbow'],
                                              ['life drain', 'longsword']]
multiattacks_df.at['Wyvern', 'multiattack'] = [['bite', 'stinger'],
                                               ['bite', 'claws'],
                                               ['claws', 'stinger']]

We note that the 'Glabrezu' and 'Violet Fungus' could be further refined.

# other actions

In [10]:
def damage_summary(damage_dice, damage_bonus):
    pattern = re.compile(r'(\d+)d(\d+)')
    damage = [np.array([damage_bonus, 0])]
    for dice in re.finditer(pattern, damage_dice):
        x, y = dice.groups()
        damage.append(damage_statistics(int(x), int(y)))
    mean, variance = sum(damage)
    return mean, variance


def damage_statistics(number, size):
    mu = (size + 1) / 2
    mean = number * mu
    var = number * (sum_of_squares(size) / size - mu ** 2)
    return np.array([mean, var])


def sum_of_squares(n):
    return n*(2*n+1)*(n+1)/6

Some of the breath weapons have to be split up.

In [11]:
actions_df.head()

desc  \
name    action                                                               
Aboleth Enslave (3/day)  The aboleth targets one creature it can see wi...   
        Multiattack              The aboleth makes three tentacle attacks.   
        Tail             Melee Weapon Attack: +9 to hit, reach 10 ft. o...   
        Tentacle         Melee Weapon Attack: +9 to hit, reach 10 ft., ...   
Acolyte Club             Melee Weapon Attack: +2 to hit, reach 5 ft., o...   

                         attack_bonus damage_dice  damage_bonus  
name    action                                                   
Aboleth Enslave (3/day)             0                         0  
        Multiattack                 0                         0  
        Tail                        9         3d6             5  
        Tentacle                    9         2d6             5  
Acolyte Club                        2         1d4             0

In [12]:
actions_df.filter(like='Lair', axis='index')

,,desc,attack_bonus,damage_dice,damage_bonus
name,action,,,,


In [13]:
desc = 'Melee Weapon Attack: +12 to hit, reach 15 ft., one target. Hit: 16 (2d8 + 7) bludgeoning damage.'
pattern = re.compile(r'^(.* Attack: .* targets?). (Hit: .* damage.)')
g = re.match(pattern, desc)

In [14]:
g.span()

(0, 96)

In [15]:
def parse_desc(desc):
    pattern = re.compile(r'^(.* Attack: .* (creature|target)s?)\. (Hit: [^\.]*)')
    g = re.match(pattern, desc)
    if g:
        start, end = g.span()
        attack, _, result = g.groups()
        print (parse_attack(attack), parse_result(result))
    else:
        pass
    return

def parse_attack(attack):
    pattern = re.compile(r'Attack: ' \
                         + r'+(\d+) to hit, ' \
                         + r'(.*)ft\.\W?, ' \
                         + r''
                        )
    return attack


def parse_result(result):
    return ''


actions_df.desc.apply(parse_desc)

Melee Weapon Attack: +9 to hit, reach 10 ft. one target 
Melee Weapon Attack: +9 to hit, reach 10 ft., one target 
Melee Weapon Attack: +2 to hit, reach 5 ft., one target 
Melee Weapon Attack: +11 to hit, reach 10 ft., one target 
Melee Weapon Attack: +11 to hit, reach 5 ft., one target 
Melee Weapon Attack: +11 to hit, reach 15 ft., one target 
Melee Weapon Attack: +12 to hit, reach 10 ft., one target 
Melee Weapon Attack: +12 to hit, reach 5 ft., one target 
Melee Weapon Attack: +12 to hit, reach 15 ft., one target 
Melee Weapon Attack: +12 to hit, reach 10 ft., one target 
Melee Weapon Attack: +12 to hit, reach 5 ft., one target 
Melee Weapon Attack: +12 to hit, reach 15 ft., one target 
Melee Weapon Attack: +11 to hit, reach,.0 ft., one target 
Melee Weapon Attack: +11 to hit, reach 5 ft., one target 
Melee Weapon Attack: +11 to hit, reach 15 ft., one target 
Melee Weapon Attack: +12 to hit, reach 10 ft., one target 
Melee Weapon Attack: +12 to hit, reach 5 ft., one target 
Melee W

name                  action                         
Aboleth               Enslave (3/day)                    None
                      Multiattack                        None
                      Tail                               None
                      Tentacle                           None
Acolyte               Club                               None
Adult Black Dragon    Acid Breath (Recharge 5-6)         None
                      Bite                               None
                      Claw                               None
                      Frightful Presence                 None
                      Multiattack                        None
                      Tail                               None
Adult Blue Dracolich  Bite                               None
                      Claw                               None
                      Frightful Presence                 None
                      Lightning Breath (Recharge 5-6)    None
                

In [16]:
for x in actions_df.iterrows():
    if x[0][1] == 'Multiattack':
        continue
    if not x[1].damage_dice:
        continue
    print(x[0])
    mean, var = damage_summary(x[1].damage_dice,
                               x[1].damage_bonus)
    print(mean, var)
    print('\n')
    # print('mean: {:0.2f}\nstd : {:0.2f}'.format(mean, np.sqrt(var)))

('Aboleth', 'Tail')
15.5 8.75


('Aboleth', 'Tentacle')
12.0 5.83333333333


('Acolyte', 'Club')
2.5 1.25


('Adult Black Dragon', 'Acid Breath (Recharge 5-6)')
54.0 63.0


('Adult Black Dragon', 'Bite')
21.5 21.75


('Adult Black Dragon', 'Claw')
13.0 5.83333333333


('Adult Black Dragon', 'Tail')
15.0 10.5


('Adult Blue Dracolich', 'Bite')
23.5 24.75


('Adult Blue Dracolich', 'Claw')
14.0 5.83333333333


('Adult Blue Dracolich', 'Lightning Breath (Recharge 5-6)')
66.0 99.0


('Adult Blue Dracolich', 'Tail')
16.0 10.5


('Adult Blue Dragon', 'Bite')
23.5 24.75


('Adult Blue Dragon', 'Claw')
14.0 5.83333333333


('Adult Blue Dragon', 'Lightning Breath (Recharge 5-6)')
66.0 99.0


('Adult Blue Dragon', 'Tail')
16.0 10.5


('Adult Brass Dragon', 'Bite')
17.0 16.5


('Adult Brass Dragon', 'Breath Weapons (Recharge 5-6)')
45.5 37.9166666667


('Adult Brass Dragon', 'Claw')
13.0 5.83333333333


('Adult Brass Dragon', 'Tail')
15.0 10.5


('Adult Bronze Dragon', 'Bite')
18.0 16.5


('Adult

('Swarm of Spiders', 'Bites')
10.0 5.0


('Swarm of Wasps', 'Bites')
10.0 5.0


('Tarrasque', 'Bite')
36.0 47.6666666667


('Tarrasque', 'Claw')
28.0 21.0


('Tarrasque', 'Horns')
32.0 33.0


('Tarrasque', 'Tail')
24.0 11.6666666667


('Thug', 'Heavy Crossbow')
5.5 8.25


('Thug', 'Mace')
5.5 2.91666666667


('Tiger', 'Bite')
8.5 8.25


('Treant', 'Rock')
28.0 33.0


('Treant', 'Slam')
16.5 8.75


('Tribal Warrior', 'Spear')
4.5 2.91666666667


('Triceratops', 'Gore')
24.0 21.0


('Triceratops', 'Stomp')
22.5 24.75


('Troll', 'Bite')
7.5 2.91666666667


('Troll', 'Claw')
11.0 5.83333333333


('Tyrannosaurus Rex', 'Bite')
33.0 47.6666666667


('Tyrannosaurus Rex', 'Tail')
20.5 15.75


('Unicorn', 'Hooves')
11.0 5.83333333333


('Unicorn', 'Horn')
8.5 5.25


('Vampire', 'Bite (Bat or Vampire Form Only)')
18.0 11.6666666667


('Vampire', 'Unarmed Strike (Vampire Form Only)')
8.5 5.25


('Vampire Spawn', 'Claws')
8.0 2.5


('Veteran', 'Heavy Crossbow')
6.5 8.25


('Veteran', 'Longsword')


In [17]:
oddities = ('Adult Brass Dragon - Bite',
            'Ancient Black Dragon - Tail')

In [18]:
damage_summary('1d6 + 2d4', 4)

(12.5, 5.4166666666666661)